In [ ]:
with open("labeled.txt","r") as f:
  text=f.read().lower().split("\n")


# Model 1 0~1 Negative conf   Model 2 0~1 Positive conf
X1 = []
Y1 = []
X2 = []
Y2 = []
for i in text:
  if i == "":
    continue
  o = i.split("------")
  print(o)
  if o[1] == "":
    X1.append(o[0])
    Y1.append(0)
    X2.append(o[0])
    Y2.append(0)

In [80]:
with open("Positive.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(o[0])
  Y1.append(0)
  X2.append(o[0])
  Y2.append(1)

In [81]:
with open("Negative.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(o[0])
  Y1.append(1)
  X2.append(o[0])
  Y2.append(0)

In [78]:
np.sum(Y1)

307

In [82]:
word_to_token = {}
token_count = 0
tokened = []

for i in X1:
  line = []
  for word in i:
    try:
      line.append(word_to_token[word])
    except:
      token_count+=1
      word_to_token[word]=token_count
      line.append(word_to_token[word])
  tokened.append(line)
max_length = 0
for i in tokened:
  if len(i)>max_length:
    max_length=len(i)

In [83]:
padded = []
for i in tokened:
  i.extend([0]*(max_length+1-len(i)))
  padded.append(i)

In [84]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length+1)),
    tf.keras.layers.Embedding(
        input_dim=token_count+2,
        output_dim=64),
    # tf.keras.layers.Conv1D(16, 5, activation='relu'),
    # tf.keras.layers.Conv1D(32, 5, activation='relu'),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GRU(50, activation='relu', return_sequences=1),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [115]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [116]:
# model.fit(X, padded)

model.fit(padded, Y1, epochs=30, shuffle=True)

Epoch 1/30
25/25 [==============================] - 26s 964ms/step - loss: 0.6796 - accuracy: 0.5995
Epoch 2/30
25/25 [==============================] - 24s 965ms/step - loss: 0.6366 - accuracy: 0.6008
Epoch 3/30
25/25 [==============================] - 24s 957ms/step - loss: 0.5788 - accuracy: 0.5839
Epoch 4/30
25/25 [==============================] - 23s 919ms/step - loss: 0.5477 - accuracy: 0.6008
Epoch 5/30
25/25 [==============================] - 25s 974ms/step - loss: 0.5414 - accuracy: 0.6216
Epoch 6/30
25/25 [==============================] - 23s 925ms/step - loss: 0.5462 - accuracy: 0.6125


KeyboardInterrupt: ignored

In [ ]:
ans = []
for i in range(200):
  ans.append(model.predict([padded[i]]))

In [73]:
model.predict([padded[8]])

1/1 [==============================] - 1s 729ms/step


array([[0.00062599]], dtype=float32)

In [ ]:
ans

In [117]:
from sklearn.feature_extraction.text import CountVectorizer

In [118]:
tf = CountVectorizer()
X = tf.fit_transform(X1)

In [119]:
X1

['major league hacking, inc ("mlh", "us" or "we") respects the privacy of the parties visiting its website, the home page of which is located at http://www.mlh.io',
 'the home page and accompanying pages will collectively be known as "website"',
 'this privacy policy is intended to educate you on our policies and practices regarding the collection, use and disclosure of any personal information (defined below) and anonymous information (defined below) you submit to us or we collect through our website',
 'unless otherwise defined herein, capitalized terms shall have the definitions assigned to such terms set forth in the mlh terms and conditions that you can find at terms and conditions and which incorporate this privacy policy "personal information" is information about you that is personally identifiable to you, like your name, address, email address, or phone number, as well as other non-public information that is associated with the foregoing',
 '“anonymous information” is informat